In [1]:
import os

def createGroupedData(distribution):
    grouped_data = dict()
    for i in range(1, 101):
        for j in range(1, 1476):
            path = os.path.join(distribution, f'trace{i}', f'{j}.Inflow')
            if os.path.exists(path):
                with open(os.path.join(path), 'r') as f:
                    lines = f.readlines()
                    # Extract the primary information, which are lines after the first three lines
                    data = [float(line.strip()) for line in lines[3:]]
                    
                    if data:  # Only append if data was successfully read
                        for k, number in enumerate(data):
                            if f"{j}.{k}" not in grouped_data:
                                grouped_data[f"{j}.{k}"] = []
                            # Append the number to the list corresponding to the key
                            grouped_data[f"{j}.{k}"].append(number)
    return grouped_data

In [2]:
grouped_data = createGroupedData("Annealing_Baseline")

In [3]:
print(grouped_data["1.0"])

[2.5958, 2.4342, 2.4478, 2.8033, 3.9525, 1.2305, 2.0804, 3.225, 3.5439, 2.0901, 1.539, 1.3926, 2.6484, 3.8412, 2.0016, 3.0552, 1.6777, 3.0949, 3.1634, 2.1699, 2.0797, 2.7663, 2.0007, 2.2234, 2.0804, 2.5957, 3.2779, 3.1521, 3.0199, 2.0856, 4.6421, 1.7843, 3.2946, 2.0413, 1.8019, 2.5958, 4.2906, 5.4161, 0.89346, 2.1225, 1.8019, 1.3763, 1.8213, 4.0937, 3.0266, 2.581, 2.3114, 4.0796, 2.6785, 2.4478, 4.231, 3.1868, 1.1017, 1.4438, 4.0729, 2.5899, 3.4886, 1.6777, 2.2234, 2.1663, 5.2997, 1.2058, 1.4871, 1.9847, 1.4814, 2.8064, 3.7514, 2.1229, 1.4758, 1.5404, 1.8435, 3.5439, 3.1052, 4.0729, 2.4342, 1.9431, 2.0019, 2.2234, 3.1868, 2.2437, 2.0267, 2.1421, 3.4924, 2.1663, 1.2572, 2.2234, 3.4924, 1.744, 3.225, 2.0405, 2.9517, 2.0856, 1.8845, 5.3366, 1.1941, 2.3764, 5.3352, 2.9534, 2.6785, 2.2588]


In [4]:
atbara = [325, 18, 45, 1356, 135, 627, 1361, 139, 545, 1381, 1322, 1326, 1352, 648, 15, 1339, 1344, 1348, 750, 1319, 1366, 765]
blueNile = [3, 1301, 484, 514, 1334, 1304, 492, 493, 474 ,1336, 1297, 452, 1058, 1296, 401, 393, 450, 441, 437, 429, 415, 412, 448, 2, 458, 463, 352, 356, 361, 343, 40, 334, 1293, 329, 247, 1, 307, 308, 1291, 293, 303, 1004, 485, 1377, 282, 230, 949, 286, 242, 234, 1253, 231, 261, 259, 255, 440, 228, 222, 1234, 1219, 248, 1287, 893, 210, 204, 209, 200, 201, 202, 425, 515, 1306]
dinder = [1369, 1308, 635, 340]
gerdToRoseires = [33, 530, 1374]
rahad = [524, 519, 243, 10]
roseiresToAbuNaama = [ 1309]
sukiToSennar = [ 470, 1364, 1317, 1381, 31]
whiteNile = [ 735, 1312, 1314, 260, 615, 21, 29, 634, 636, 1475, 715, 729, 807, 645, 794, 786, 608, 792, 1353, 1331, 779, 787, 17, 30, 1311, 9, 748, 739, 163, 162, 170, 178, 182, 150, 23, 176, 177, 349, 193, 190, 153, 355, 158, 184, 727, 26, 577, 737, 731, 420, 872, 420 ]

In [5]:
regions = {
    "atbara": atbara, 
    "blueNile" :blueNile, 
    "dinder": dinder, 
    "gerdToRoseires": gerdToRoseires, 
    "rahad": rahad, 
    "roseiresToAbuNaama": roseiresToAbuNaama, 
    "sukiToSennar": sukiToSennar, 
    "whiteNile": whiteNile
}

nu_of_days_per_month = [
            31,  # January
            28,  # February (non-leap year)
            31,  # March
            30,  # April
            31,  # May
            30,  # June
            31,  # July
            31,  # August
            30,  # September
            31,  # October
            30,  # November
            31,  # December
        ]

In [6]:
print(len(grouped_data.values()))

97200


In [8]:
import numpy as np

for x in regions:
    with open(f"Averages/{x}.txt", "w") as f:
        for a in range(83, 323):
            summ = 0 
            for y in regions[x]:
                summ += np.mean(grouped_data[f"{y}.{a}"])
            perSecondAvg = summ * 1000000 / (nu_of_days_per_month[(a + 1) % 12] * 86400.0) # +1 because the measurements start january 31 24:00
            f.write(f"{perSecondAvg}\n")

In [13]:
from tqdm import tqdm

distributions = {
    "Annealing_Baseline": "Baseline_grouped",
    "Annealing_120Hurst" : "120Hurst_grouped",
    "Annealing_150ias" : "150ias_grouped",
}

for x in tqdm(distributions.keys()):
    grouped_data = createGroupedData(x)
    os.mkdir(f"{distributions[x]}")
    for trace in range(100):
        os.mkdir(f"{distributions[x]}/trace{trace}")
        for reg in regions:
            with open(os.path.join(f"{distributions[x]}/trace{trace}/{reg}.txt"), "w") as f:
                for k in range(83, 323):
                    summ = 0
                    for y in regions[reg]:
                        summ += grouped_data[f"{y}.{k}"][trace]
                    avgPerSec = summ * 1000000 / (nu_of_days_per_month[(a + 1) % 12] * 86400.0) 
                    f.write(f"{avgPerSec}\n")   

100%|██████████| 3/3 [01:04<00:00, 21.36s/it]
